In [25]:
try:
    from openmdao.utils.notebook_utils import notebook_mode  # noqa: F401
except ImportError:
    !python -m pip install openmdao[notebooks]

# What is Julia?

[Julia](https://julialang.org/) is a new-ish programming language targeting scientific and high-performance computing applications.
Like Python or MATLAB, it is a dynamic language—you don't need to declare the types of variables like you would in C, C++, Fortran, etc..
Unlike Python, it is a compiled language, as every function is compiled to machine code the first time it is called.
The aim of Julia is to provide users with a programming language that combines the convience of Python or MATLAB with the speed of C or Fortran.
The [official Julia documentation](https://docs.julialang.org/en/v1/) has [a nice page comparing Julia to other popular languages](https://docs.julialang.org/en/v1/manual/noteworthy-differences/) if you'd like to know more.

One of the most attractive features of Julia for OpenMDAO users is its good support for automatic differentiation (AD).
There are many (perhaps too many!) AD libraries available for the language, all with particular strengths and weaknesses.
Happily, an excellent library called [DifferentiationInterface.jl](https://github.com/JuliaDiff/DifferentiationInterface.jl) exists which allows us to use any of these numerious AD library via a uniform API, which makes switching from one to another a matter of changing just one or two lines of code.

# What is OpenMDAO.jl?
[OpenMDAO.jl](https://github.com/byuflowlab/OpenMDAO.jl) is a Python+Julia software package that allows users to create OpenMDAO `Component`s out of Julia code, and optionally use a Julia AD package to differentiate the `Component`.
There are currently two possible approaches to using AD with Julia:
1. Manual Approach: Users can write the equivalent of a `compute_partials` or `linearize` method in Julia that calls the desired AD library manually. This is quite flexible and conceptually straightforward, but requires more work on the part of the user to manage the complexities of translating the `inputs` and `outputs` vectors OpenMDAO provides to something that the AD library can work with.
2. Automatic Approach: Users can write a relatively simple "callback function" that implements the math behind the `Component` they wish to create, and allow OpenMDAO.jl to call the AD for them. This is much less work for the user, but does require learning a bit more about the Julia language. This approach is also currently limited to creating `ExplicitComponent`s (though the callback function can contain implicitness via the [ImplicitAD.jl](https://github.com/byuflowlab/ImplicitAD.jl) package).

This documentation will focus on approach number 2, the Automatic Approach, which consists of essentially three steps:

1. Create a user-defined callback function
2. Create an OpenMDAO.jl `AbstractADExplicitComp` `struct` that will keep track the user-defined function, the desired AD library, and other optional features like units, tags, etc..
3. Pass the `AbstractADExplicitComp` to omjlcomps ("OpenMDAO Julia Components"), a Python library that knows how to create OpenMDAO `Component`s from an OpenMDAO.jl `AbstractADExplicitComp`.

After completing those three steps, you'll be left with an OpenMDAO `Component` that you can include anywhere in an OpenMDAO model.
We'll go through each of the three steps below. 
For more details, have a look at the [OpenMDAO.jl docs](https://flow.byu.edu/OpenMDAO.jl/dev/).

# Guided Tutorial

## Step 1: The User-Defined Function
The user-defined function *must* follow one of two forms: either it can be an "in-place" function that writes its outputs to an output vector, or it can be an "out-of-place" function that returns a single output vector.
The function must be written in Julia, of course, and anything that happens inside the function is up to you, of course.
Both the in-place and out-of-place form must also have a `params` argument that will contain inputs that are needed for the calculation, but won't be differentiated (the equivalent of OpenMDAO `options`).
So, an example of an in-place function would be

```julia
function f_in_place!(Y, X, params)
   # calculate stuff with X and params, storing result in Y
   return nothing
end
```

where `X` is the input vector and `Y` is the output vector.
(The function doesn't have to return `nothing`, but any returned value will be ignored, so I like to include `return nothing` to make it clear that the return value doesn't matter.)
An out-of-place function would look like

```julia
function f_out_of_place(X, params)
   # calculate stuff with X and params, returning Y
   return Y
end
```

where again `X` is the input vector and `Y` is the output vector.

Now, the `X` and `Y` arguments of those functions must not be plain Julia `Vector`s, but `ComponentVectors` from the [ComponentArrays.jl](https://github.com/jonniedie/ComponentArrays.jl) package.
What are those?
They are objects provided by the [ComponentArrays.jl](https://github.com/SciML/ComponentArrays.jl) package that act like `Vector`s, but allow the user to define names for each part ("component") of the vector.
For example, we can do this from the Julia REPL:

```julia
julia> using ComponentArrays: ComponentVector

julia> x1 = ComponentVector(foo=-1.0, bar=-2.0, baz=-3.0)
ComponentVector{Float64}(foo = -1.0, bar = -2.0, baz = -3.0)

julia> @show x1 x1[3] x1.foo x1[:foo]
x1 = (foo = -1.0, bar = -2.0, baz = -3.0)
x1[3] = -3.0
x1.foo = -1.0
x1[:foo] = -1.0
-1.0

julia> 
```

Notice that we can get, say, the third value of `x1` the usual way (`x1[3]`), but also by referring to the `foo` field value via `x1.foo` and by indexing the `ComponentVector` with the symbol `:foo` (`x1[:foo]`).

Each of the components in `x1` are scalars, but they don't have to be:

```julia
julia> x2 = ComponentVector(foo=-1.0, bar=1:4, baz=reshape(5:10, 2, 3))
ComponentVector{Float64}(foo = -1.0, bar = [1.0, 2.0, 3.0, 4.0], baz = [5.0 7.0 9.0; 6.0 8.0 10.0])

julia> @show x2 x2[:foo] x2[:bar] x2[:baz]
x2 = (foo = -1.0, bar = [1.0, 2.0, 3.0, 4.0], baz = [5.0 7.0 9.0; 6.0 8.0 10.0])
x2[:foo] = -1.0
x2[:bar] = [1.0, 2.0, 3.0, 4.0]
x2[:baz] = [5.0 7.0 9.0; 6.0 8.0 10.0]
2×3 Matrix{Float64}:
 5.0  7.0   9.0
 6.0  8.0  10.0

julia> 
```

In `x2`, the `foo` component is a scalar, `bar` refers to a `Vector` (aka a 1D `Array`) and `baz` refers to a `Matrix` (aka a 2D Array).
But `x2` still "looks like" a `Vector`:

```julia
julia> @show x2[3]  # will give the third value of `x2`, which happens to be the second value of x2[:bar]
x2[3] = 2.0
2.0

julia> @show ndims(x2)  # Should be 1, since a Vector is 1-dimensional
ndims(x2) = 1
1

julia> @show length(x2)  # length(x2) gives the total number of entries in `x2`, aka 1 + 4 + 2*3 = 11
length(x2) = 11
11

julia> @show size(x2)  # size is a length-1 tuple since a Vector has just one dimension
size(x2) = (11,)
(11,)

julia> 
```

Now, how will we use `ComponentVector`s here?
We'll use them to define the names and sizes of all the inputs and outputs to our component.
For example, if we wanted to implement [the Paraboloid Example](../../basic_user_guide/single_disciplinary_optimization/first_analysis), which minimizes the function

$$
f(x,y) = (x-3.0)^2 + x \cdot y + (y+4.0)^2 - 3.0
$$

using OpenMDAO.jl, we could create an input vector called `X_ca` that looks like this:

```julia
julia> X_ca = ComponentVector(x=1.0, y=1.0)
ComponentVector{Float64}(x = 1.0, y = 1.0)

julia> 
```

That has two scalar components, `x` and `y`, that correspond to the two inputs to the paraboloid function.
We could similarly use an output vector `Y_ca` that looks like this:

```julia
julia> Y_ca = ComponentVector(f_xy=0.0)
ComponentVector{Float64}(f_xy = 0.0)

julia> 
```

with the single output `f_xy`.

We could type in a callback function that implements the paraboloid equation above into the REPL if we want:

```julia
julia> function f_paraboloid!(Y, X, params)
           # Get the inputs:
           x = @view(X[:x])
           y = @view(X[:y])
           # Could also do this:
           # x = X.x
           # y = X.y
           # or even this
           # (; x, y) = X

           # Get the output:
           f_xy = @view(Y[:f_xy])
           # Again, could also do this:
           # f_xy = Y.f_xy
           # or
           # (; f_xy) = Y

           # Do the calculation:
           @. f_xy = (x - 3.0)^2 + x*y + (y + 4.0)^2 - 3.0

           # No return value for in-place callback function.
           return nothing
       end
f_paraboloid! (generic function with 1 method)

julia> 
```

And then test it out with the `X_ca` and `Y_ca` `ComponentVector`s we created earlier:

```julia
julia> f_paraboloid!(Y_ca, X_ca, nothing)

julia> X_ca
ComponentVector{Float64}(x = 1.0, y = 1.0)

julia> Y_ca
ComponentVector{Float64}(f_xy = 27.0)

julia> 
```

Some remarks about the callback function:

* The `@view` macro is used when extracting the inputs and outputs from the `X_ca` and `Y_ca` `ComponentVectors`. This creates a view into the original ComponentVector, instead of a new array with a copy of the original data, which avoids unnecessary allocations and (for the outputs) allows modifications to the view to be reflected in the `Y_ca` array. In this example everything is a scalar, so no allocations would have happened anyway. But it doesn't hurt to use `@view`: it's a good habit to get into, and it allows us to use the `@.` broadcasting macro with the scalar `f_xy` output.
* The `params` argument is not used in this example, but it is still required, since the code in OpenMDAO.jl will expect it. We provided a `nothing` value for it, but we could have given it anything and not changed the result of the function, of course.

An out-of-place version of `f_paraboloid!` would look like this:

```julia
julia> function f_paraboloid(X, params)
           x = @view(X[:x])
           y = @view(X[:y])
           f_xy = @. (x - 3.0)^2 + x*y + (y + 4.0)^2 - 3.0
           return ComponentVector(f_xy=f_xy)
       end
f_paraboloid (generic function with 1 method)

julia> f_paraboloid(X_ca, nothing)
ComponentVector{Float64}(f_xy = 27.0)

julia> 
```

But we only need one or the other.

## Step 2: The OpenMDAO.jl `AbstractADExplicitComp`
So far we've done a lot of typing in the Julia REPL, which is great for testing, but not great for writing code that you'll use more than once.
Where should we store this Julia code that we're creating, and how will we call it from Python?
This section will explain my favorite way to do that.
Along the way we'll learn how to choose a Julia AD library, and create an `AbstractADExplicitComp` that will eventually be used to instantiate an `ExplicitComponent` that can be added to a OpenMDAO model.

### A Julia Package for our Paraboloid
We'll store our Julia code for the paraboloid component in a brand new Julia package called MyParaboloidPackage.
Creating a new Julia package is very easy from the REPL: just type `]` to enter the [Pkg REPL](https://pkgdocs.julialang.org/v1/repl/) (look for the `pkg>` in the prompt to ensure you're in "Pkg mode"), then type `generate MyParaboloidPackage` to create the package:

```julia
(openmdao_jl_dev) pkg> generate MyParaboloidPackage
  Generating  project MyParaboloidPackage:
    MyParaboloidPackage/Project.toml
    MyParaboloidPackage/src/MyParaboloidPackage.jl

(openmdao_jl_dev) pkg> 
```

(Keep this REPL open, as we'll be using it again later.)
As the output suggests, running that command creates a directory called `MyParaboloidPackage` in the current working directory, along with two files.
The `Project.toml` file contains metadata about the Julia project (name, version number, a UUID to distinguish it from other Julia packages, dependencies, etc.).
The `src/MyParaboloidPackage.jl` contains the source code of the package, and at the moment looks like this:

```julia
module MyParaboloidPackage

greet() = print("Hello World!")

end # module MyParaboloidPackage
```

We'll put our paraboloid code in `src/MyParaboloidPackage.jl` by copy-pasting the definition of `f_paraboloid!` into the file.
Now it should look like this (with the `greet` function removed):

```julia
module MyParaboloidPackage

function f_paraboloid!(Y, X, params)
    x = @view(X[:x])
    y = @view(X[:y])
    f_xy = @view(Y[:f_xy])
    @. f_xy = (x - 3.0)^2 + x*y + (y + 4.0)^2 - 3.0
    return nothing
end

end # module MyParaboloidPackage
```

Now, let's see if we can create that `AbstractADExplicitComp` that we've been talking about.
There are a few different flavors of `AbstractADExplicitComp` `struct`s that we could use, but for this example, we'll use `DenseADExplicitComp`, one that's designed for explicit components with a dense Jacobian.
If we check out the [documentation for `DenseADExplicitComp` constructor](https://flow.byu.edu/OpenMDAO.jl/dev/reference/#OpenMDAOCore.DenseADExplicitComp-Union{Tuple{TAD},%20Tuple{TAD,%20Any,%20ComponentArrays.ComponentVector,%20ComponentArrays.ComponentVector}}%20where%20TAD%3C:ADTypes.AbstractADType), we'll see that we need four things to get this done:

* An `ad_backend`, to be explained shortly
* A callback function `f!`
* `Y_ca`, a `ComponentVector` of outputs
* `X_ca`, a `ComponentVector` of inputs

In the previous section we learned how to create `X_ca` and `Y_ca`, and our callback function `f!` will be the `f_paraboloid!`.
So the only missing ingrediant is the `ad_backend`.

The `ad_backend` is a Julia object that will tell the `DenseADExplicitComp` which AD library we want to use to differentiate our component.
The object must be a subtype of `AbstractADType`, an abstract type provided by the [ADTypes.jl](https://github.com/SciML/ADTypes.jl) package.
That package is not part of the Julia Standard Library, so we'll need to install it.
How do we do that?
The easiest way is from the Julia Pkg REPL, aka the thing we used to generate the `MyParaboloidPackage` earlier.
First, type `]` to ensure we're in the Pkg REPL (again, look for `pkg>` in the prompt), then do this:

```
(openmdao_jl_dev) pkg> activate ./MyParaboloidPackage
  Activating project at `~/projects/openmdao_jl_dev/ved/OpenMDAO/openmdao/docs/openmdao_book/features/experimental/MyParaboloidPackage`

(MyParaboloidPackage) pkg> 
```

That "activates" the environment associated with MyParaboloidPackage, which tells the Julia Pkg package manager that we want to mess around with the dependencies of it and not some other environment or package.
(The Pkg REPL reminds us which environment is "active" by changing the prompt to `(MyParaboloidPackage) pkg>`.) 
We can check the status of MyParaboloidPackage via the `status` command:

```julia
(MyParaboloidPackage) pkg> status
Project MyParaboloidPackage v0.1.0
Status `~/projects/openmdao_jl_dev/ved/OpenMDAO/openmdao/docs/openmdao_book/features/experimental/MyParaboloidPackage/Project.toml` (empty project)

(MyParaboloidPackage) pkg> 
```

Unsuprisingly there isn't much going on with it yet.

The next step is to add the dependencies we need.
We've already decided that we want ADTypes.jl, but we'll also need:

* ComponentArrays.jl: for the `ComponentVector`s we'll provide to the `DenseADExplicitComp` constructor
* OpenMDAOCore.jl: for the `DenseADExplicitComp` itself
* ForwardDiff.jl: a forward-mode AD library that we'll use for this example

So, let's install those using the `add` command from the Pkg REPL:

```julia
(MyParaboloidPackage) pkg> add ADTypes ComponentArrays OpenMDAOCore ForwardDiff
   Resolving package versions...
   Installed DifferentiationInterface ─ v0.7.14
      Compat entries added for ADTypes, ComponentArrays, OpenMDAOCore, ForwardDiff
    Updating `~/projects/openmdao_jl_dev/ved/OpenMDAO/openmdao/docs/openmdao_book/features/experimental/MyParaboloidPackage/Project.toml`
  [47edcb42] + ADTypes v1.21.0
  [b0b7db55] + ComponentArrays v0.15.31
  [f6369f11] + ForwardDiff v1.3.1
  [24d19c10] + OpenMDAOCore v0.3.3
    Updating `~/projects/openmdao_jl_dev/ved/OpenMDAO/openmdao/docs/openmdao_book/features/experimental/MyParaboloidPackage/Manifest.toml`

# Lots of output removed for clarity
(MyParaboloidPackage) pkg> 
```

Now if we try the `status` command again, we'll see this:

```julia
(MyParaboloidPackage) pkg> status
Project MyParaboloidPackage v0.1.0
Status `~/projects/openmdao_jl_dev/ved/OpenMDAO/openmdao/docs/openmdao_book/features/experimental/MyParaboloidPackage/Project.toml`
  [47edcb42] ADTypes v1.21.0
  [b0b7db55] ComponentArrays v0.15.31
  [f6369f11] ForwardDiff v1.3.1
  [24d19c10] OpenMDAOCore v0.3.3

(MyParaboloidPackage) pkg> 
```

That shows the dependencies of `MyParaboloidPackage`, and their version numbers.
Excellent.

Now we should have everything we need to create the `DenseADExplicitComp`.
I think it's easiest to create a small "helper function" that we can call later that returns our `DenseADExplicitComp` that looks like this:

```julia
function get_paraboloid_comp()
    ad_backend = ADTypes.AutoForwardDiff()
    X_ca = ComponentVector(x=1.0, y=1.0)
    Y_ca = ComponentVector(f_xy=0.0)
    comp = OpenMDAOCore.DenseADExplicitComp(ad_backend, f_paraboloid!, Y_ca, X_ca)
    return comp
end
```

The `ADTypes.AutoForwardDiff()` means that we'll use [ForwardDiff.jl](https://github.com/JuliaDiff/ForwardDiff.jl), a forward-mode AD package, to differentiate our paraboloid.

We'll add that function to `src/MyParaboloidPackage.jl`, along with some import statements that give us access to the packages we just installed.
So now it should look like this:

```julia
module MyParaboloidPackage

using ADTypes: ADTypes
using ComponentArrays: ComponentVector
using ForwardDiff: ForwardDiff
using OpenMDAOCore: OpenMDAOCore

function f_paraboloid!(Y, X, params)
    x = @view(X[:x])
    y = @view(X[:y])
    f_xy = @view(Y[:f_xy])
    @. f_xy = (x - 3.0)^2 + x*y + (y + 4.0)^2 - 3.0
    return nothing
end

function get_paraboloid_comp()
    ad_backend = ADTypes.AutoForwardDiff()
    X_ca = ComponentVector(x=1.0, y=1.0)
    Y_ca = ComponentVector(f_xy=0.0)
    comp = OpenMDAOCore.DenseADExplicitComp(ad_backend, f_paraboloid!, Y_ca, X_ca)
    return comp
end

end # module MyParaboloidPackage
```

We can try out the `get_paraboloid_comp` function from the normal Julia REPL (not the Pkg REPL—just backspace until you get the `julia> ` prompt back):

```julia
julia> using MyParaboloidPackage

julia> comp = MyParaboloidPackage.get_paraboloid_comp()
```

You'll probibly see a bunch of output describing in excruciating detail all the type information associated with the `DenseADExplicitComp` we just created.
But as long as you see something that starts with `OpenMDAOCore.DenseADExplicitComp`, we should be good.


## Step 3: The `omjlcomps.JuliaExplicitComp`
So far we have created a Julia callback function and a `DenseADExplicitComp` for our paraboloid.
The last step is to turn that `DenseADExplicitComp` into something we can actually incorporate into an OpenMDAO model.
We'll do that using omjlcomps, a Python package that's part of OpenMDAO.jl.

But first, we need to figure out a nice way of telling Python about our Julia code.
The best way of doing that is using [juliapkg](https://github.com/JuliaPy/pyjuliapkg), a Python package that allows us to add Julia dependencies to a Python package.
We could install it manually ourselves using `pip install juliapkg`, but if instead we install OpenMDAO with the "julia" extra, we'll get it automatically:

```bash
$ pip install openmdao[julia]
```

We'll do that from here in a Jupyter notebook using this command:

In [26]:
!pip install openmdao[julia]

Now, we need to use juliapkg to tell our Python environment about MyParaboloidPackage.
We do that by creating a file called `juliapkg.json` in our working directory that points to MyParaboloidPackage:

```json
{"packages": {
    "MyParaboloidPackage": {"uuid": "a013c6e3-dc76-40b3-894d-62618f778e16", "dev": true, "path": "./MyParaboloidPackage"}
    }
}
```

(To get the correct value for the `"uuid"` field, copy what's in the `MyParaboloidPackage/Project.toml` file.
It will be different from what is listed above if you're trying out these commands yourself.)

That tells juliapkg about MyParaboloidPackage.
The `"dev": true` means that the package will be installed in "develop" mode, meaning any changes in the package will be reflected in code that calls it—it's the equivalent of `pip install -e` or `pip install --editable` in Python.

`juliapkg` searches for `juliapkg.json` files in, among other places, each entry in `sys.path`.
This is great for adding Julia dependencies to Python packages: you can add a `juliapkg.json` to your Python source, and `juliapkg` will find it automatically if your Python package is installed in the current environment.
It also works well for Python scripts run from the command line: since Python automatically adds the directory of the script to `sys.path`, we could place the `juliapkg.json` file in the same directory as the run script and it should also be found by `juliapkg`.
But it's not-so-great for Jupyter notebooks like this one.
So we'll have to manually add what we think is the current working directory to `sys.path` and cross our fingers.


In [27]:
import os
import sys
d = os.getcwd()
sys.path.append(d)
print(sys.path[-1])

/home/dingraha/projects/openmdao_jl_dev/ved/OpenMDAO/openmdao/docs/openmdao_book/features/experimental


Next, we'll create a small julia file in the current working directory that will import the `get_paraboloid_comp` function that we just added to `MyParaboloidPackage` called `paraboloid.jl`:

In [28]:
display(Code(os.path.join(d, "paraboloid.jl")))

using MyParaboloidPackage: get_paraboloid_comp

Now we'll create another small file, this time a Python one, called `paraboloid.py` that will include that `paraboloid.jl` julia file using JuliaCall, a Python package for calling Julia code that was automatically installed when we did `pip install openmdao[julia]`:

In [29]:
display(Code(os.path.join(d, "paraboloid.py")))

import os

# Create a new Julia module that will hold all the Julia code imported into this Python module.
import juliacall; jl = juliacall.newmodule("ParaboloidComponentsStub")

# Get the directory this file is in, then include the `paraboloid.jl` Julia source code.
d = os.path.dirname(os.path.abspath(__file__))
jl.include(os.path.join(d, "paraboloid.jl"))
# Now we have access to everything in `paraboloid.jl` in the `jl` object.

get_paraboloid_comp = jl.get_paraboloid_comp

Now we should be able to use omjlcomps (another Python package that was installed as a side effect of `pip install openmdao[julia]`) to create a `JuliaExplicitComp`.
We do this by first calling `get_paraboloid_comp` to get the `DenseADExplicitComp` `struct` we created, then pass it as the `jlcomp` argument to `JuliaExplicitComp`:

In [30]:
import omjlcomps
from paraboloid import get_paraboloid_comp
jlcomp = get_paraboloid_comp()
comp = omjlcomps.JuliaExplicitComp(jlcomp=jlcomp)

Now we have a OpenMDAO `Component` that we can use.
Let's try it out, following the [Paraboloid example](../../basic_user_guide/single_disciplinary_optimization/first_analysis) from the OpenMDAO docs:

In [31]:
import openmdao.api as om

model = om.Group()
model.add_subsystem('parab_comp', comp)

prob = om.Problem(model)

prob.model.add_design_var("parab_comp.x")
prob.model.add_design_var("parab_comp.y")
prob.model.add_objective("parab_comp.f_xy")

prob.driver = om.ScipyOptimizeDriver()
prob.driver.options["optimizer"] = "SLSQP"

prob.setup(force_alloc_complex=True)

prob.set_val("parab_comp.x", 3.0)
prob.set_val("parab_comp.y", -4.0)

prob.run_model()
print(prob["parab_comp.f_xy"])  # Should print `[-15.]`

prob.set_val("parab_comp.x", 5.0)
prob.set_val("parab_comp.y", -2.0)

prob.run_model()
print(prob.get_val("parab_comp.f_xy"))  # Should print `[-5.]`

/home/dingraha/projects/openmdao_jl_dev/ved/OpenMDAO/openmdao/visualization/n2_viewer/n2_viewer.py:115: OpenMDAOWarning:'float' object has no attribute 'shape'


-15.0
-5.0


Looks good so far.
A few things to note:
* We added the `JuliaExplicitComp` to the `Group` in the normal way.
* The variable names exposed to OpenMDAO match what was used in the input and output `ComponentVector`s passed to the `DenseADExplicitComp` constructor.
* `prob.set_val` and `prob.get_val` worked just like usual.

We can also check our derivatives using the usual OpenMDAO methods. We can do that with the finite difference method:

In [32]:
print(prob.check_partials(method="fd"))

------------------------------------------------------------------------------
Component: JuliaExplicitComp 'parab_comp'
------------------------------------------------------------------------------

  parab_comp: 'f_xy' wrt 'x'

    Max Tolerance Violation (Jfwd - Jfd) - (atol + rtol * Jfd) : (-9.996325e-07)
      abs error: 1.000368e-06
      rel error: 5.001840e-07
      fwd value @ max viol: 2.000000e+00
      fd value @ max viol: 2.000001e+00 (fd:forward)

    Raw Forward Derivative (Jfwd)
    [[ 2.000000000000e+00]]

    Raw FD Derivative (Jfd)
    [[ 2.000001000368e+00]]

 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

  parab_comp: 'f_xy' wrt 'y'

    Max Tolerance Violation (Jfwd - Jfd) - (atol + rtol * Jfd) : (-7.999542e-06)
      abs error: 1.000459e-06
      rel error: 1.111621e-07
      fwd value @ max viol: 9.000000e+00
      fd value @ max viol: 9.000001e+00 (fd:forward)

    Raw Forward Derivative (Jfwd)
    [[ 9.000000000000e+00]]

    Raw FD Derivative (Jfd)
    [[ 9.000001000459e+00]]

 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

{'parab_comp': {('f_xy', 'x'): {'J_fwd': array([[2.]]), 'J_fd': array([[2.000001]]), 'rows': None, 'cols': None, 'tol violation': _ErrorData(forward=-9.99632543852158e-07, reverse=None, fwd_rev=None), 'magnitude': _MagnitudeData(forward=2.0, reverse=0.0, fd=2.0000010003684565), 'vals_at_max_error': _ErrorData(forward=(np.float64(2.0), np.float64(2.0000010003684565)), reverse=None, fwd_rev=None), 'abs error': _ErrorData(forward=1.0003684565162985e-06, reverse=None, fwd_rev=None), 'rel error': _ErrorData(forward=5.001839780740122e-07, reverse=None, fwd_rev=None)}, ('f_xy', 'y'): {'J_fwd': array([[9.]]), 'J_fd': array([[9.000001]]), 'rows': None, 'cols': None, 'tol violation': _ErrorData(forward=-7.999542276593274e-06, reverse=None, fwd_rev=None), 'magnitude': _MagnitudeData(forward=9.0, reverse=0.0, fd=9.000001000458724), 'vals_at_max_error': _ErrorData(forward=(np.float64(9.0), np.float64(9.000001000458724)), reverse=None, fwd_rev=None), 'abs error': _ErrorData(forward=1.000458723865449

Or we could use the complex-step method:

In [33]:
print(prob.check_partials(method="cs"))

------------------------------------------------------------------------------
Component: JuliaExplicitComp 'parab_comp'
------------------------------------------------------------------------------

  parab_comp: 'f_xy' wrt 'x'

    Max Tolerance Violation (Jfwd - Jfd) - (atol + rtol * Jfd) : (-2.000000e-06)
      abs error: 0.000000e+00
      rel error: 0.000000e+00
      fwd value @ max viol: 2.000000e+00
      fd value @ max viol: 2.000000e+00 (cs:None)

    Raw Forward Derivative (Jfwd)
    [[ 2.000000000000e+00]]

    Raw CS Derivative (Jfd)
    [[ 2.000000000000e+00]]

 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

  parab_comp: 'f_xy' wrt 'y'

    Max Tolerance Violation (Jfwd - Jfd) - (atol + rtol * Jfd) : (-9.000000e-06)
      abs error: 0.000000e+00
      rel error: 0.000000e+00
      fwd value @ max viol: 9.000000e+00
      fd value @ max viol: 9.000000e+00 (cs:None)

    Raw Forward Derivative (Jfwd)
    [[ 9.000000000000e+00]]

    Raw CS Derivative (Jfd)
    [[ 9.000000000000e+00]]

 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

{'parab_comp': {('f_xy', 'x'): {'J_fwd': array([[2.]]), 'J_fd': array([[2.]]), 'rows': None, 'cols': None, 'tol violation': _ErrorData(forward=-2e-06, reverse=None, fwd_rev=None), 'magnitude': _MagnitudeData(forward=2.0, reverse=0.0, fd=2.0), 'vals_at_max_error': _ErrorData(forward=(np.float64(2.0), np.float64(2.0)), reverse=None, fwd_rev=None), 'abs error': _ErrorData(forward=0.0, reverse=None, fwd_rev=None), 'rel error': _ErrorData(forward=0.0, reverse=None, fwd_rev=None)}, ('f_xy', 'y'): {'J_fwd': array([[9.]]), 'J_fd': array([[9.]]), 'rows': None, 'cols': None, 'tol violation': _ErrorData(forward=-9e-06, reverse=None, fwd_rev=None), 'magnitude': _MagnitudeData(forward=9.0, reverse=0.0, fd=9.0), 'vals_at_max_error': _ErrorData(forward=(np.float64(9.0), np.float64(9.0)), reverse=None, fwd_rev=None), 'abs error': _ErrorData(forward=0.0, reverse=None, fwd_rev=None), 'rel error': _ErrorData(forward=0.0, reverse=None, fwd_rev=None)}}}


The derivatives look great, so let's try an optimization:

In [34]:
prob.run_driver()
print(f"f_xy = {prob.get_val("parab_comp.f_xy")}")  # Should print `[-27.333333]`
print(f"x = {prob.get_val("parab_comp.x")}")  # Should print `[6.666666]`
print(f"y = {prob.get_val("parab_comp.y")}")  # Should print `[-7.333333]`

Optimization terminated successfully    (Exit mode 0)
            Current function value: -27.333333333333336
            Iterations: 4
            Function evaluations: 5
            Gradient evaluations: 4
Optimization Complete
-----------------------------------
f_xy = -27.333333333333336
x = [6.66666667]
y = [-7.33333333]


Victory!

# Next Steps
We've shown how to put together a simple OpenMDAO `Component` using Julia and OpenMDAO.jl.
OpenMDAO.jl has more features that you can [explore in docs](https://flow.byu.edu/OpenMDAO.jl/dev/).
Some things we haven't covered:
* We just used one AD package in the example above (ForwardDiff.jl), but there are many more. Check out the [ADTypes.jl documentation](https://docs.sciml.ai/ADTypes/stable/) for all the possible options.
* We used the `DenseADExplicitComp` `struct` to create our Paraboloid component, which is the most appropriate for a component with a dense Jacobian. But OpenMDAO.jl can also create components that evaluate the derivatives using the [matrix-free approach](https://flow.byu.edu/OpenMDAO.jl/dev/matrix_free_ad/), or [accelerate the calculation of sparse Jacobians using coloring algorithms](https://flow.byu.edu/OpenMDAO.jl/dev/auto_sparse_ad/).
* The OpenMDAO.jl docs also have [recommendations on how to create Python packages that contain Julia OpenMDAO components](https://flow.byu.edu/OpenMDAO.jl/dev/creating_python_packages/) that you should definitely check out if you plan on using OpenMDAO.jl for "real work." There is also an example Python package that follows these guidelines in the GitHub repository.
* In the aformentioned example Python package in the OpenMDAO.jl GitHub repository there is an example of using OpenMDAO.jl and [ImplicitAD.jl](https://github.com/byuflowlab/ImplicitAD.jl) to solve [the nonlinear circuit example from the OpenMDAO docs](../../advanced_user_guide/models_implicit_components/models_with_solvers_implicit). This shows how we can use OpenMDAO.jl to efficiently differentiate through an implicit calculation with a little help from ImplicitAD.jl.